This notebook explores what happens to the Energy Function $x^2 + y^2 + z^2$ for different numerical solvers to the Lorenz System. This is checked for three different parameter tripples and for some different initial conditions.

In [ ]:
using Plots
using LinearAlgebra
using DifferentialEquations
using Random
using QuadGK

In [ ]:
include("euler_method.jl")
include("DTM.jl")
include("lorentz_system.jl")
include("RK4.jl")
include("EulerCromer.jl")

First we have some help functions for our analysis

In [ ]:
function EnergyFunction(x)
    # a very simple energy function
    return x[1]^2 + x[2]^2 + x[3]^2 
end

In [ ]:
# Finds the energy at each step of the trajectory for starting position x̄₀
function Energy_of_Path(ẋ::Function, Δt, tim, x̄₀, p)
    tsteps = Int(tim/Δt)
    x̄=x̄₀
    Energy = zeros(tsteps)
    for i=1:tsteps
        x̄= ẋ(p,x̄,Δt)
        Energy[i] = EnergyFunction(x̄)
    end
    return Energy
end

In [ ]:
# Creates nr_of_dots initial conditions on a sphere with radius r
function Points_on_Sphere(nr_of_dots, r)
    dots = zeros(3, nr_of_dots)
    for i=1:nr_of_dots
        dot = rand(-1.0:0.001:1.0,(1,3))
        dot = r*dot./norm(dot)
        dots[:,i] = dot
    end
    return dots
end

In [ ]:
function Plot_Energies(ẋ,initial_vals, time, Δt,p)
    Energies = zeros(length(initial_vals[1,:]), Int(time/Δt))
    i=1
    for x̄₀ in eachcol(initial_vals)
        Energies[i,:] = Energy_of_Path(ẋ, Δt, time, x̄₀, p)
        i+=1
    end

    plt=plot(
        yaxis= :log10,
        title="Semilog plot of the Energy",
        xlabel="time",
        ylabel="Energy"
    )
    plot(plt,range(0,stop=time-Δt,step=Δt),Energies[1,:])
    plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[2,:])
    plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[3,:])
    plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[4,:])
    plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[5,:])
    #plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[6,:])
    #plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[7,:])
    #plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[8,:])
    #plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[9,:])
    #plot!(plt,range(0,stop=time-Δt,step=Δt),Energies[10,:])
end

In [ ]:
prob = ODEProblem(LorentzSystem,[1,2,3],(0.0,time),[θ,μ,β])
sol = solve(prob, RadauIIA5(), dt=Δt)
plot(sol.t,[EnergyFunction(x) for x in sol.u])

In [ ]:
function Plot_Energies_Preex(ẋ,initial_vals, time, Δt,p)

    Energies = zeros(length(initial_vals[1,:]), Int(time/Δt))
    solutions = [[]]
    i=1
    p=plot(
        yaxis= :log10,
        title="Semilog plot of the Energy",
        xlabel="time",
        ylabel="Energy"
    )
    for x̄₀ in eachcol(initial_vals)
        prob = ODEProblem(LorentzSystem,x̄₀,(0.0,time),[θ,μ,β])
        sol = solve(prob, ẋ, dt=Δt)
        NRG = zeros(length(sol.u[:,1]))
        plot!(p,sol.t,[EnergyFunction(x) for x in sol.u],label=string([round(x̄₀[1]),round(x̄₀[2]),round(x̄₀[3])]),legend = :outertopleft)
        solutions = [solutions, sol.t]
        i+=1
    end
    display(p)
end

Analysis of the energy when $\mu < 1$ and the origin is globally stable

In [ ]:
## parameter values
θ = 10.0; β = 8/3; μ = 1/2
time = 20
Δt = 0.001

# initial values
nr_of_initial_conditions = 5
r = 250 # From the analysis of global stability, we know that all solvers converge to the origin when r <= 250
initial_vals = Points_on_Sphere(nr_of_initial_conditions, r)

In [ ]:
Plot_Energies_Preex(RadauIIA5(), initial_vals, time, Δt, [θ,μ,β])
savefig("RadauIIA5Energy_origin.png")

In [ ]:
#Euler Forward
Plot_Energies(EulerForward, initial_vals, time, Δt, [θ,μ,β])
savefig("EulerForwardEnergy_origin.png")

In [ ]:
#Runge Kutta 4
Plot_Energies(RK4, initial_vals, time, Δt, [θ,μ,β])
savefig("RK4Energy_origin.png")

In [ ]:
#DTM
Plot_Energies(DTM, initial_vals, time, Δt, [θ,μ,β])
savefig("DTMEnergy_origin.png")

In [ ]:
Plot_Energies(EulerCromer, initial_vals, time, Δt, [θ,μ,β])
savefig("EulerCromerEnergy_origin.png")

Analysis of the energy for when there exists two additional fixpoints to the origin

In [ ]:
## parameter values
θ = 10.0; β = 8/3; μ = 23.5
time = 100
Δt = 0.001

# Fixed points
x̄_fix1 = [sqrt(8*22.5/3);
          sqrt(8*22.5/3);
          22.5]

x̄_fix2 = [-sqrt(8*22.5/3);
          -sqrt(8*22.5/3);
          22.5]

In [ ]:
# Euler Forward
# Basically didn't want to converge for Δt=0.001
Δt = 0.0001
nr_of_initial_conditions = 5
r = 1/100 # the fix points are not globally stable but stable
initial_vals = Points_on_Sphere(nr_of_initial_conditions, r)
initial_vals1 = initial_vals .+ x̄_fix1
Plot_Energies(EulerForward, initial_vals1, time, Δt, [θ,μ,β])
savefig("EulerForwardEnergy_fix1.png")

In [ ]:
nr_of_initial_conditions = 5
Δt = 0.001

r =2 # the fix points are not globally stable but stable
initial_vals = Points_on_Sphere(nr_of_initial_conditions, r)
initial_vals1 = initial_vals .+ x̄_fix1
initial_vals2 = initial_vals .+ x̄_fix2

In [ ]:
#Runge Kutta 4
Plot_Energies(RK4, initial_vals1, time, Δt, [θ,μ,β])
savefig("RK4_fixp.png")

In [ ]:
Plot_Energies_Preex(RadauIIA5(), initial_vals1, time, Δt, [θ,μ,β])
savefig("RadauIIA5Energy_fix1.png")

In [ ]:
# DTM
Plot_Energies(DTM, initial_vals1, time, Δt, [θ,μ,β])
savefig("DTMEnergy_fix1.png")

In [ ]:
# Euler Cromer
Plot_Energies(EulerCromer, initial_vals1, time, Δt, [θ,μ,β])
savefig("EulerCromerEnergy_fix1.png")

Analysis of the energy for the traditional chaotic Lorenz parameters $\theta=10$, $\beta = \frac{8}{3}$, $\mu = 28$

In [ ]:
## parameter values
θ = 10.0; β = 8/3; μ = 28
time = 50
Δt = 0.001

# initial values
nr_of_initial_conditions = 5
r = 250 # From the analysis of global stability, we know that all solvers converge to the origin when r <= 250
intitial_vals = Points_on_Sphere(nr_of_initial_conditions, r)

In [ ]:
Plot_Energies_Preex(RadauIIA5(), initial_vals1, time, Δt, [θ,μ,β])
savefig("RadauIIA5Energy_chaos.png")

In [ ]:
#Euler Forward
Plot_Energies(EulerForward, initial_vals, time, Δt, [θ,μ,β])
savefig("EulerForwardEnergy_chaos.png")

In [ ]:
#Runge Kutta 4
Plot_Energies(RK4, initial_vals, time, Δt, [θ,μ,β])
savefig("RK4Energy_chaos.png")

In [ ]:
#DTM
Plot_Energies(DTM, initial_vals, time, Δt, [θ,μ,β])
savefig("DTMEnergy_chaos.png")

In [ ]:
# Euler Cromer
Plot_Energies(EulerCromer, initial_vals, time, Δt, [θ,μ,β])
savefig("EulerCromerEnergy_chaos.png")